In [1]:
import sqlite3


In [2]:
import importlib

import pandas as pd
import optuna
import gc
import copy

import sys
sys.path.append("../nlp_assist/")

In [3]:
def d2p(d, f):
    with open(f, 'wb') as file:
        pickle.dump(d, file)

def p2d(f):
    with open(f, 'rb') as file:
        d = pickle.load(file)
    return d

def d2j(d, f):
    with open(f, "wt") as file:
        json.dump(d, file, indent=2)

def j2d(f):
    with open(f, "rt") as file:
        d = json.load(file)
    return d

In [4]:
import datetime
import pickle
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
ts = now.strftime('%Y%m%d_%H%M%S_%f')
ts

'20260103_185922_057477'

In [5]:
#カラム内の文字数。デフォルトは50だった
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", 101)

In [6]:
#条件もとの3テーブルを作るモジュール
import nlp_connect as nl
importlib.reload(nl)

#各種条件をquboとして作るためのモジュール
import nlp_limiter as lmt
importlib.reload(lmt)

#並列で探索するソルバのラッパー
import p_solver as ps
importlib.reload(ps)

#結果チェッカ等
import res_parser as rps
importlib.reload(rps)

import nsp_settings as ns
importlib.reload(ns)

print("reloaded!")
f = "../demos/dmk0.3_20220111_21.xlsx"
f2 = "../demos/dmk0.3_20220111_21add.xlsx"

x = nl.xlsx2condtion(f)
x.make3CDT(year=2020, month= 11)

# q_genはqubo generator
qg = lmt.q_gen(x2c=x)

dummy var generator activated
start status is 0
limter activated!
dummy var generator activated
start status is 0
limter activated!
reloaded!
xlsx file is
../demos/dmk0.3_20220111_21.xlsx
make condition tables.
holidaymode: asHoliday
year:  2020
month:  11
holidaymode:asHoliday
days that is holiday and not sunday will be Holiday.
Q generator activated!
x2c need to pre make 3DT
nsp_expression activated!


In [7]:
f_df = "./optuna_study_itr_whole_fix_lchs.db"

con = sqlite3.connect(f_df)

In [8]:
def q2df(q):
    df = pd.DataFrame(con.execute(q))
    
    len_col = len(df.columns)
    df.columns = ["c"+str(i+1) for i in range(len_col)]
    return df

In [9]:
df_t = q2df("select name from sqlite_master where type='table';")

In [10]:
df_t

,c1
0,studies
1,version_info
2,study_directions
3,study_user_attributes
4,study_system_attributes
5,trials
6,trial_user_attributes
7,trial_system_attributes
8,trial_params
9,trial_values


In [11]:
df_trial = q2df("select * from trial_values")\
    .query("c4 == 1")
df_trial


,c1,c2,c3,c4,c5
770,771,775,0,1.0,FINITE
818,819,825,0,1.0,FINITE
851,852,857,0,1.0,FINITE
941,942,949,0,1.0,FINITE
1087,1088,1090,0,1.0,FINITE
1115,1116,1120,0,1.0,FINITE
1124,1125,1133,0,1.0,FINITE
1195,1196,1197,0,1.0,FINITE
1228,1229,1233,0,1.0,FINITE
1230,1231,1238,0,1.0,FINITE


In [12]:
tirals_one = df_trial["c2"]

In [13]:
df_param = q2df("select * from trial_params") \
    .query("c2 in @tirals_one")

# .query()
df_param.head()

,c1,c2,c3,c4,c5
13932,13933,775,1by1,146.0,"{""name"": ""IntDistribution"", ""attributes"": {""log"": false, ""step"": 1, ""low"": 1, ""high"": 256}}"
13933,13934,775,cnight,101.0,"{""name"": ""IntDistribution"", ""attributes"": {""log"": false, ""step"": 1, ""low"": 1, ""high"": 256}}"
13934,13935,775,cwork,5.0,"{""name"": ""IntDistribution"", ""attributes"": {""log"": false, ""step"": 1, ""low"": 1, ""high"": 256}}"
13935,13936,775,dab,75.0,"{""name"": ""IntDistribution"", ""attributes"": {""log"": false, ""step"": 1, ""low"": 1, ""high"": 256}}"
13936,13937,775,fseq,187.0,"{""name"": ""IntDistribution"", ""attributes"": {""log"": false, ""step"": 1, ""low"": 1, ""high"": 256}}"


In [14]:
df_param_sub = df_param \
    .loc[:, ["c2", "c3", "c4"]]
# d_params.head().to_dict()
d_params = dict()
for r in df_param_sub.itertuples():
    if r.c2 not in d_params.keys():
        d_params[r.c2] = dict()
    d_params[r.c2][r.c3] = r.c4

In [15]:
d_params[775]

{'1by1': 146.0,
 'cnight': 101.0,
 'cwork': 5.0,
 'dab': 75.0,
 'fseq': 187.0,
 'lcnn': 92.0,
 'lcnt': 103.0,
 'lfbd': 146.0,
 'lnd': 168.0,
 'mps': 122.0,
 'nnv': 105.0,
 'nseq': 145.0,
 'rqh': 14.0,
 'rqn': 183.0,
 'sc': 64.0,
 'tn': 1.0,
 'tt': 23.0,
 'sweeps': 473.0}

In [16]:
import importlib

import pandas as pd
import optuna
import gc
import copy

import sys
sys.path.append("../nlp_assist/")

import pandas as pd
import optuna
import gc
import copy

#条件もとの3テーブルを作るモジュール
import nlp_connect as nl
importlib.reload(nl)

#各種条件をquboとして作るためのモジュール
import nlp_limiter as lmt
importlib.reload(lmt)

#並列で探索するソルバのラッパー
import p_solver as ps
importlib.reload(ps)

#結果チェッカ等
import res_parser as rps
importlib.reload(rps)

import nsp_settings as ns
importlib.reload(ns)

print("reloaded!")


dummy var generator activated
start status is 0
limter activated!
reloaded!


In [17]:
f = "../demos/dmk0.3_20220111_21.xlsx"
f2 = "../demos/dmk0.3_20220111_21add.xlsx"
# x = nl.xlsx2condtion(f)

In [18]:

import json

def d2j(d, f):
    with open(f, "wt") as file:
        json.dump(d, file, indent=2)

def j2d(f):
    with open(f, "rt") as file:
        d = json.load(file)
    return d


In [19]:
f_param = "../note05_lmt_debug/e0param.json"
d_param = j2d(f_param)

In [20]:
d_param

{'1by1': 166.0,
 'cnight': 98.0,
 'cwork': 19.0,
 'dab': 109.0,
 'fseq': 226.0,
 'lcnn': 39.0,
 'lcnt': 99.0,
 'lfbd': 160.0,
 'lnd': 143.0,
 'mps': 127.0,
 'nnv': 107.0,
 'nseq': 167.0,
 'rqh': 31.0,
 'rqn': 197.0,
 'sc': 85.0,
 'tn': 1.0,
 'tt': 11.0,
 'sweeps': 819.0}

In [21]:
inc = ('1by1','cnight',
     'cwork',
     'dab',
    #  'eqg2',
    #  'eqg3',
     'fseq',
     'lcnn',
     'lcnt',
     'lfbd',
     'lnd',
     'mps',
     'nnv',
     'nseq',
     'rqh',
     'rqn',
     'sc',
     'tn',
     'tt')

In [22]:
res_dict = dict()

In [23]:
%%time

iter_num = 100
for k in range(iter_num):
    print(k)
    print(d_param)
    swmul = d_param["sweeps"]
    param_dict = d_param.copy()
    # sw_base = param_dict["sweeps"]
    del param_dict["sweeps"]
    
    # sw_base = 1e+2
    sw_base = 1e+4
    sw = int(sw_base *swmul )
    x = nl.xlsx2condtion(f)
    x.make3CDT(year=2020, month=11)

    # q_genはqubo generator
    qg = lmt.q_gen(x2c=x)
    qg.setParam(param_dict)
    print(param_dict)
    qg.makeAllLimitation(do_normalize=True, hideProgressBar=True, include=inc)

    p = ps.psol(q=qg.ne.getQ(),
                 num_reads=1,
                 sweeps=sw,
                 seed=k)
    mins = p.p_solve_getMin(threads=21, times=21)

    rp = rps.res_parser(x=x)
    rp.prepRes(res=mins.sample, hideProgressBar=True)
    rc = rps.res_checker(x=x, echo_bool=True)
    rc.checkAllLimitation(res_parser=rp, include=inc)
    colp_num = len(rc.log_df)
    tm = rps.table_maker(x=x, res=mins.sample)


    res_dict_k = dict()
    res_dict_k["errornum"] = len(rc.log_df)
    res_dict_k["mins_sample"] = mins.sample
    res_dict_k["log_df"] = rc.log_df
    res_dict_k["timetable"]= tm.datetable.copy()
    res_dict_k["shifttable"] = tm.shifttable.copy()

    print("trial tag:" + str(k))
    print("error num:" + str(res_dict_k["errornum"]))
    print("colappse:" +str(colp_num))

    f_pkl = "./res_dict_202011_e{}_k{}_ts{}.pkl".format(res_dict_k["errornum"], k, ts)
    print(f_pkl)

    d2p(d=res_dict_k, f=f_pkl)

    break

0
{'1by1': 166.0, 'cnight': 98.0, 'cwork': 19.0, 'dab': 109.0, 'fseq': 226.0, 'lcnn': 39.0, 'lcnt': 99.0, 'lfbd': 160.0, 'lnd': 143.0, 'mps': 127.0, 'nnv': 107.0, 'nseq': 167.0, 'rqh': 31.0, 'rqn': 197.0, 'sc': 85.0, 'tn': 1.0, 'tt': 11.0, 'sweeps': 819.0}
xlsx file is
../demos/dmk0.3_20220111_21.xlsx
make condition tables.
holidaymode: asHoliday
year:  2020
month:  11
holidaymode:asHoliday
days that is holiday and not sunday will be Holiday.
Q generator activated!
x2c need to pre make 3DT
nsp_expression activated!
{'1by1': 166.0, 'cnight': 98.0, 'cwork': 19.0, 'dab': 109.0, 'fseq': 226.0, 'lcnn': 39.0, 'lcnt': 99.0, 'lfbd': 160.0, 'lnd': 143.0, 'mps': 127.0, 'nnv': 107.0, 'nseq': 167.0, 'rqh': 31.0, 'rqn': 197.0, 'sc': 85.0, 'tn': 1.0, 'tt': 11.0}
Q dict is resetted!
one can do one task per day...
limit continuous work by dt1 and member capacity
limit continious night work by dt3...
forbiden sequence is forbiden..
must pair must nor...
limitting as req numbers
limitting as req work ti

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:01<00:00, 24.86it/s]


checking continenous nights works...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:01<00:00, 19.78it/s]


checking is there forbiden seq
clear!
checking needed pair is pair...
clear
number is correct?
clear
is total hour correct?
clear
is total work times is correct?
clear
is total night is correct?
clear
is forbid seq is avoided?
ﾆB Yy
NN Yy
Yy ﾆB
Yy NN
clear
is needed seq is preserved?
MY Yy
clear
checking work count from last month...
clear
checking night work count from last month
clear
checking shift count satisfication
clear
checking forbidden pair in shifts
clear
checking daytime veteran groups
clear
checking night time novice groups
clear
table_maker activated.
parser activated!
checking...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7949/7949 [00:00<00:00, 2649648.14it/s]
D:\anal\nsp_kh\note06_opt_fixed_param\../nlp_assist\res_parser.py:1578: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  t3_wid = t3_lim.pivot_table(values=['duty_str'], index=['date_number', "num_days"], columns=['name'],aggfunc=lambda x: ' '.join(x)).T\
D:\anal\nsp_kh\note06_opt_fixed_param\../nlp_assist\res_parser.py:1597: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  dt3x = pd.merge(df_Final_sub,
D:\anal\nsp_kh\note06_opt_fixed_param\../nlp_assist\res_parser.py:1597: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  dt3x = pd.merge(df_Final_sub,
100%|████████████████████████████

trial tag:0
error num:0
colappse:0
./res_dict_202011_e0_k0_ts20260103_185922_057477.pkl
Wall time: 1h 19min 59s


D:\anal\nsp_kh\note06_opt_fixed_param\../nlp_assist\res_parser.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt1.loc[:, "weekdays"] = dt1["num_days"].copy().apply(
D:\anal\nsp_kh\note06_opt_fixed_param\../nlp_assist\res_parser.py:1688: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt1.loc[:, "holiday_count"] = dt1["isHoliday"].copy().apply(
D:\anal\nsp_kh\note06_opt_fixed_param\../nlp_assist\res_parser.py:1692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [24]:
kokomade

NameError: name 'kokomade' is not defined